In [1]:
import pandas as pd
import numpy as np
import ast  # To parse string representations of lists and dictionaries safely
import os 
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

# Set Pandas options to display full text for all columns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  # Show all columns


In [28]:
# Load the datasets from the './data' folder
companies_path = './data/YC F24 Founder Company.csv'
people_path = './data/YC F24 Founder People.csv'

# Read data into pandas DataFrames
companies_data = pd.read_csv(companies_path)
people_data = pd.read_csv(people_path)

# Relevant columns to keep
relevant_columns = [
    'full_name', 'title', 'location', 'company', 'company_linkedin', 
    'company_description', 'sub_title', 'summary', 'current_job_description', 
    'person_industry', 'skills', 'education', 'work_history']

# Filter the relevant columns
companies_filtered = companies_data[relevant_columns]
people_filtered = people_data[relevant_columns]

import re

# Helper function to extract the "name" field using a simple regex
def extract_names_simple(data_str):
    try:
        # Find all occurrences of "name": "value"
        names = re.findall(r'"name":\s?"(.*?)"', data_str)
        return ", ".join(names) if names else "No Names Found"
    except Exception as e:
        return f"Error: {str(e)}"

# Apply the updated function to the education and work history columns
def preprocess_columns_simple(df):
    df['education'] = df['education'].apply(extract_names_simple)
    df['work_history'] = df['work_history'].apply(extract_names_simple)
    return df

# Apply to both datasets
companies_preprocessed = preprocess_columns_simple(companies_filtered.copy())
people_preprocessed = preprocess_columns_simple(people_filtered.copy())


# Display the filtered datasets
print("Filtered Companies Data:")
display(companies_preprocessed.head())

print("Filtered People Data:")
display(companies_preprocessed.head())


Filtered Companies Data:


,full_name,title,location,company,company_linkedin,company_description,sub_title,summary,current_job_description,person_industry,skills,education,work_history
0,Edward Aryee,Co-Founder & CTO,"San Francisco Bay, San Francisco Bay Area, United States of America",SRE.ai (YC F24),https://www.linkedin.com/company/104394435/,NaN,Co-Founder & CTO at SRE.ai (YC F24) | ex-Google,NaN,An AI-powered automation platform for Salesforce development teams.,Software Development,"[""Programming"", ""Python"", ""Java"", ""JavaScript"", ""Matlab"", ""Computer Science"", ""Public Speaking""]",Drexel University,"SRE.ai (YC F24), Black Angel Group (BAG), Google, Microsoft, StratIS IoT, Web Presence in China, BuLogics, Virtual Pantry, Columbia University"
1,Victor Cheng,Co-Founder and CEO,"San Francisco, California, United States of America",vly.ai (YC F24),https://www.linkedin.com/company/99356232/,NaN,co-founder @ vly.ai (YC F24),cooking,The easiest way to build custom full-stack software without code. Backed by Y Combinator,Software Development,"[""Startups"", ""TypeScript"", ""Git"", ""Express.js"", ""MongoDB"", ""Full-Stack Development"", ""Node.js"", ""Competitive Programming"", ""Next.js"", ""Software Development"", ""React.js"", ""C++"", ""JavaScript"", ""C#"", ""Java"", ""Unity3D"", ""Business"", ""Mathematics"", ""Robotics"", ""Mobile Application Development"", ""Android Studio""]","University of Washington, Skyline High School, Y Combinator","vly.ai (YC F24), Hatch Coding, Minute Land, Quantum Labs, Autodesk, FRC Team 2976, Spartabots, Self-employed"
2,Daryl Budiman,Co-Founder & CEO,"San Francisco Bay, San Francisco Bay Area, United States of America",Andoria (YC F24),https://www.linkedin.com/company/102953213/,NaN,Co-Founder at Andoria (YC F24) | Prev. MultiOn & Stanford,Imagine if you didn't have to learn any piece of software anymore.,"Backed by YC, among other great investors.","Technology, Information and Internet","[""Customer Onboarding"", ""Customer Success"", ""SwiftUI"", ""C++"", ""React.js"", ""Microsoft PowerPoint"", ""HTML"", ""CSS"", ""JavaScript"", ""Python"", ""Presentation Skills"", ""Public Speaking"", ""Adobe Photoshop"", ""Research"", ""ArcGIS"", ""Figma"", ""Leadership"", ""User Interface Design"", ""Computer-Aided Design (CAD)"", ""Autodesk Fusion 360""]","Stanford University, Y Combinator","Andoria (YC F24), MultiOn, Amazon, The Marriage Pact, Evernote, Matcha, Stanford University School of Medicine"
3,Finn Mallery,Co-Founder,"Palo Alto, California, United States of America",Origami Agents (YC F24),https://www.linkedin.com/company/99440945/,NaN,Co-Founder @ Origami Agents (YC F24),NaN,NaN,"Technology, Information and Internet",[],"Stanford University, Stanford University School of Engineering, Z Fellows, Canyon Crest Academy","Origami Agents (YC F24), Fizz, StartX., Stanford Marketing"
4,Arvind V.,Co-Founder and CEO,"San Francisco, California, United States of America",Fresco (YC F24),https://www.linkedin.com/company/104799979/,NaN,Building Fresco (YC F24) | Acquired Founder,"I’m currently building Fresco (YC F24) an AI copilot for construction superintendents. \n\nPreviously, I was Chief of Staff and Director of Partnerships at January AI, a precision medicine company based in the Bay Area. I have my BA and MS from Brown, and I've also worked at NASA (SynBio), Human Capital (VC), and Vault Health (PM).\n\nI enjoyed an exit from my political tech startup, DownBallot Solutions, in 2022. I also make investments in early-stage technology companies, with a focus on consumer software and digital health. I'm always happy to chat about science or entrepreneurship!",NaN,Construction,"[""B2B sales"", ""Financial Modeling"", ""Strategic Partnerships"", ""Crispr"", ""Report Writing"", ""Grant Writing"", ""Cell Culture"", ""Stem Cell Research"", ""Synthetic Biology"", ""Laboratory Management"", ""Due Diligence"", ""Investments"", ""Early-stage Startups"", ""Communication"", ""Funnel Optimization"", ""Agile Methodologies"", ""Scr

Filtered People Data:


,full_name,title,location,company,company_linkedin,company_description,sub_title,summary,current_job_description,person_industry,skills,education,work_history
0,Edward Aryee,Co-Founder & CTO,"San Francisco Bay, San Francisco Bay Area, United States of America",SRE.ai (YC F24),https://www.linkedin.com/company/104394435/,NaN,Co-Founder & CTO at SRE.ai (YC F24) | ex-Google,NaN,An AI-powered automation platform for Salesforce development teams.,Software Development,"[""Programming"", ""Python"", ""Java"", ""JavaScript"", ""Matlab"", ""Computer Science"", ""Public Speaking""]",Drexel University,"SRE.ai (YC F24), Black Angel Group (BAG), Google, Microsoft, StratIS IoT, Web Presence in China, BuLogics, Virtual Pantry, Columbia University"
1,Victor Cheng,Co-Founder and CEO,"San Francisco, California, United States of America",vly.ai (YC F24),https://www.linkedin.com/company/99356232/,NaN,co-founder @ vly.ai (YC F24),cooking,The easiest way to build custom full-stack software without code. Backed by Y Combinator,Software Development,"[""Startups"", ""TypeScript"", ""Git"", ""Express.js"", ""MongoDB"", ""Full-Stack Development"", ""Node.js"", ""Competitive Programming"", ""Next.js"", ""Software Development"", ""React.js"", ""C++"", ""JavaScript"", ""C#"", ""Java"", ""Unity3D"", ""Business"", ""Mathematics"", ""Robotics"", ""Mobile Application Development"", ""Android Studio""]","University of Washington, Skyline High School, Y Combinator","vly.ai (YC F24), Hatch Coding, Minute Land, Quantum Labs, Autodesk, FRC Team 2976, Spartabots, Self-employed"
2,Daryl Budiman,Co-Founder & CEO,"San Francisco Bay, San Francisco Bay Area, United States of America",Andoria (YC F24),https://www.linkedin.com/company/102953213/,NaN,Co-Founder at Andoria (YC F24) | Prev. MultiOn & Stanford,Imagine if you didn't have to learn any piece of software anymore.,"Backed by YC, among other great investors.","Technology, Information and Internet","[""Customer Onboarding"", ""Customer Success"", ""SwiftUI"", ""C++"", ""React.js"", ""Microsoft PowerPoint"", ""HTML"", ""CSS"", ""JavaScript"", ""Python"", ""Presentation Skills"", ""Public Speaking"", ""Adobe Photoshop"", ""Research"", ""ArcGIS"", ""Figma"", ""Leadership"", ""User Interface Design"", ""Computer-Aided Design (CAD)"", ""Autodesk Fusion 360""]","Stanford University, Y Combinator","Andoria (YC F24), MultiOn, Amazon, The Marriage Pact, Evernote, Matcha, Stanford University School of Medicine"
3,Finn Mallery,Co-Founder,"Palo Alto, California, United States of America",Origami Agents (YC F24),https://www.linkedin.com/company/99440945/,NaN,Co-Founder @ Origami Agents (YC F24),NaN,NaN,"Technology, Information and Internet",[],"Stanford University, Stanford University School of Engineering, Z Fellows, Canyon Crest Academy","Origami Agents (YC F24), Fizz, StartX., Stanford Marketing"
4,Arvind V.,Co-Founder and CEO,"San Francisco, California, United States of America",Fresco (YC F24),https://www.linkedin.com/company/104799979/,NaN,Building Fresco (YC F24) | Acquired Founder,"I’m currently building Fresco (YC F24) an AI copilot for construction superintendents. \n\nPreviously, I was Chief of Staff and Director of Partnerships at January AI, a precision medicine company based in the Bay Area. I have my BA and MS from Brown, and I've also worked at NASA (SynBio), Human Capital (VC), and Vault Health (PM).\n\nI enjoyed an exit from my political tech startup, DownBallot Solutions, in 2022. I also make investments in early-stage technology companies, with a focus on consumer software and digital health. I'm always happy to chat about science or entrepreneurship!",NaN,Construction,"[""B2B sales"", ""Financial Modeling"", ""Strategic Partnerships"", ""Crispr"", ""Report Writing"", ""Grant Writing"", ""Cell Culture"", ""Stem Cell Research"", ""Synthetic Biology"", ""Laboratory Management"", ""Due Diligence"", ""Investments"", ""Early-stage Startups"", ""Communication"", ""Funnel Optimization"", ""Agile Methodologies"", ""Scr

In [2]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)


[TextBlock(text="Tears of ancient earth,\nMinerals dissolved in time,\nRivers carry salt to sea -\nNature's endless brine.\n\nThrough eons deep and vast,\nEach wave a salty kiss,\nFrom rocks worn down at last\nTo make this briny bliss.", type='text')]
